In [1]:
import numpy as np
from os import listdir
from os.path import isfile, join
import json

### Charge les matchs

In [2]:
Matchs = list(np.load("MATCHDATA\KR_matchs_data.npy",allow_pickle=True)) + list(np.load("MATCHDATA\EU_matchs_data.npy",allow_pickle=True)) + list(np.load("MATCHDATA\\NA_matchs_data.npy",allow_pickle=True))

In [3]:
len(Matchs)

348600

### Tunnel de vectorisation des champions

In [7]:
f = open('CHAMPS\\norm_vector_space.json',encoding="utf8")
Champions = json.load(f)
Champions['FiddleSticks'] = Champions.pop('Fiddlesticks')

In [11]:
def replace_name_by_vectors(matchs):
    vectored = []
    c=0
    for m in matchs:
        try:
            b_i_top,b_i_jgl,b_i_mid,b_i_adc,b_i_sup = m[0].index('TOP'), m[0].index('JUNGLE'), m[0].index('MIDDLE'), m[0].index('BOTTOM'), m[0].index('UTILITY')
            r_i_top,r_i_jgl,r_i_mid,r_i_adc,r_i_sup = m[1].index('TOP'), m[1].index('JUNGLE'), m[1].index('MIDDLE'), m[1].index('BOTTOM'), m[1].index('UTILITY')
            Blue = [Champions[m[0][b_i_top+1]],Champions[m[0][b_i_jgl+1]],Champions[m[0][b_i_mid+1]],Champions[m[0][b_i_adc+1]],Champions[m[0][b_i_sup+1]]]
            Red = [Champions[m[1][r_i_top+1]],Champions[m[1][r_i_jgl+1]],Champions[m[1][r_i_mid+1]],Champions[m[1][r_i_adc+1]],Champions[m[1][r_i_sup+1]]]
            vectored.append([Blue,Red,m[2]])
        except Exception as e:
            #print(e)
            pass
    return vectored

In [12]:
Vectored = replace_name_by_vectors(Matchs)

In [13]:
len(Vectored)

339179

### Creation jeu entraînement/test à partir des données vectorisées

In [14]:
def make_train_test(vectored):
    X_train,Y_train = [],[]
    X_test,Y_test = [],[]
    for i in range(len(vectored)):
        if i%10==0 or i%10==1:
            X_test.append([vectored[i][0],vectored[i][1]])
            Y_test.append(1) if vectored[i][2]=='Blue' else Y_test.append(0)
        else:
            X_train.append([vectored[i][0],vectored[i][1]])
            Y_train.append(1) if vectored[i][2]=='Blue' else Y_train.append(0)
    return(np.array(X_train),np.array(Y_train),np.array(X_test),np.array(Y_test))

In [15]:
X_train,Y_train,X_test,Y_test = make_train_test(Vectored)

In [19]:
len(X_train),len(X_test)

(271343, 67836)

In [20]:
np.save("SETS\X_train.npy",X_train)
np.save("SETS\X_test.npy",X_test)
np.save("SETS\Y_train.npy",Y_train)
np.save("SETS\Y_test.npy",Y_test)